# Batch scheduling of a fermentation process

Industrial fermentation processes are generally batch operations. The process consists of a
number of phases or stages, each of which requires a considerable amount of time. Because
of the time factor, the system cannot react quickly to variations in demand, so that the
scheduling of production in each stage is of considerable importance.

Consider a fermentation process which has five stages: (1) Mixing and cooking [$M$,$m$]; (2) Fermentation [$F$,$f$]; (3) Purification [$P$,$p$]; (4) Blending and packaging [$B$,$b$]; (5) Warehousing [$W$,$w$].

$m_i$ ($f_i$, $p_i$, etc.) is the quantity that goes into stage $m$ (from the previous stage) in day $i$. $d_i$ is the demand (by customers) from the warehouse in period $i$. The demand $d_i$ is known (but not precisely). Given $d_i$, a set of values $m_i$, $f_i$, $p_i$, $b_i$, $w_i$ constitutes a production schedule. The capital letters indicate the total quantity in the stage, i.e. $M_j$ is the total amount of material in the mixing stage in period $j$.

### A note on unit selection
It is a good idea to select the unit you will be using from the start, especially considering factors of 10 (such as kg, tonnes etc.). In this example, we will always use $1000L$ as our unit.

## Initialization and variable selection
As you may expect, we have to introduce variables for each step of the process, both for the lower case value (ingoing) as well as the upper case value (amount contained).

> Note that due to 0-indexing, we have $0\leq i\leq N-1$.

In [44]:
import xpress as xp
%env XPRESS=..
import numpy as np
import matplotlib.pyplot as plt
import math

model = xp.problem("Batch scheduling")

# This is required due to 1 indexing instead of 0 indexing
n = 105
nRange = range(n)
t_m = 3
A_max = 8
Z_b = 5
t_min_fermentation = 10
t_max_fermentation = 30
t_min_purification = 2
t_max_purification = 5
C_m = 350; mu = 2;
C_f = 410; phi = 5;
C_p = 380; pi = 7;
C_b = 250; beta = 2;
C_w = 200; chi = 1;

# This is the demand. We only need it later but it is good practice to define it beforehand.
d = np.array([0 for i in range(20)])
#d = np.append(d,np.array([3 for i in range(N-20)]))
#d = np.append(d,np.array([4 for i in range(N-20)]))
#d = np.append(d,np.array([4.5 for i in range(N-20)]))
d = np.append(d, np.array([5+math.sin(i/10) for i in range(n-20)]))

In [45]:
def get_variable_dictionary(name:str):
    return {day : xp.var(vartype = xp.continuous, lb = 0, name = f'{name}_{day}') for day in nRange}

m = get_variable_dictionary('Mixing inflow')
M = get_variable_dictionary('Mixing volume')

f = get_variable_dictionary('Fermentation inflow')
F = get_variable_dictionary('Fermentation volume')

p = get_variable_dictionary('Purification inflow')
P = get_variable_dictionary('Purification volume')

b = get_variable_dictionary('Blending inflow')
B = get_variable_dictionary('Blending volume')

w = get_variable_dictionary('Warehousing inflow')
W = get_variable_dictionary('Warehousing')

model.addVariable(m,M,f,F,p,P,b,B,w,W)

> Note: We package away the dictionary creation because it is just 'noise' (and the same for all the variables). Also, since we only have a single numeric index, we stick with that and do not introduce a custom class.

### Linking variables together
It is often a good idea to get the "logical" constraints out of the way first, i.e. the constraints that link variable $A$ and $B$. In this case, we link the "inflow" variables with the overall quantities.

For example, let's consider the fermentation process. Mathematically, this can be described as:

$F_j = \sum \limits_{i=0}^{j} (f_{i} - p_{i})$,

where $F_j$ is the total quanitity in the fermentation at period $j$. Similarly, we can define:

$M_j = \sum \limits_{i=0}^{j} (m_{i} - f_{i})$

$P_j = \sum \limits_{i=0}^{j} (p_{i} - b_{i})$

$B_j = \sum \limits_{i=0}^{j} (b_{i} - w_{i})$

$W_j = \sum \limits_{i=0}^{j} (w_{i} - d_{i})$

Therefore, putting this into Xpress, we get:

In [46]:
mixing_balance = (xp.constraint(M[j] == xp.Sum(m[i] - f[i] 
                                               for i in range(j+1)), 
                                name=f'Balance for mixing at day {j}') 
                  for j in nRange)
fermentation_balance = (xp.constraint(F[j] == xp.Sum(f[i] - p[i] for i in range(j+1)), 
                                      name=f'Balance for fermentation at day {j}') for j in nRange)
purification_balance = (xp.constraint(P[j] == xp.Sum(p[i] - b[i] for i in range(j+1)), 
                                        name=f'Balance for purification at day {j}') for j in nRange)
blending_balance = (xp.constraint(B[j] == xp.Sum(b[i] - w[i] for i in range(j+1)), 
                                    name=f'Balance for blending at day {j}') for j in nRange)
warehousing_balance = (xp.constraint(W[j] == xp.Sum(w[i] - d[i] for i in range(j+1)), 
                                       name=f'Balance for warehousing at day {j}') for j in nRange)

model.addConstraint(mixing_balance, fermentation_balance, purification_balance, blending_balance, warehousing_balance)

In [2]:
%env XPRESS = ..

env: XPRESS=..


In [5]:
pwd

'C:\\Users\\ricob'